# Write and read data in tfrecord format

This notebook shows how to read and write TFRecord files.

This is based on the examples [convert_to_records.py](https://github.com/tensorflow/tensorflow/blob/r1.12/tensorflow/examples/how_tos/reading_data/convert_to_records.py) and [fully_connected_reader.py](https://github.com/tensorflow/tensorflow/blob/r1.12/tensorflow/examples/how_tos/reading_data/fully_connected_reader.py) on the TensorFlow repository, and on [this post](http://warmspringwinds.github.io/tensorflow/tf-slim/2016/12/21/tfrecords-guide/).

More info can be found on [TensorFlow's home page](https://www.tensorflow.org).

In [ ]:
import matplotlib.pyplot as plt
import skimage.io as io
import tensorflow as tf

In [ ]:
cat_array = io.imread('imgs/orange-cat.jpg')
io.imshow(cat_array)
io.imshow(cat_array)
plt.title('image size=(%sx%sx%s)' % cat_array.shape)
plt.xticks([])
plt.yticks([])
plt.show()

cat_array = io.imread('imgs/gray-cat.jpg')
io.imshow(cat_array)
io.imshow(cat_array)
plt.title('image size=(%sx%sx%s)' % cat_array.shape)
plt.xticks([])
plt.yticks([])
plt.show()

### 1. Define the types that can be written into the TFRecord file

In [ ]:
def _bytes_feature(value):
    """Use this for string and byte types"""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Use this for bool, int32, uint32, int64 and uint64 types"""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _float_feature(value):
    """Use this for float (float32) and double (float64) types"""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

In [ ]:
_float_feature(4.5)

In [ ]:
_float_feature(4.5).SerializeToString()

### 2. Write the images and labels to a TFRecord file
The steps to write on a TFRecord file are:
 1. Create (open) a TFRecord file.
 2. Loop over the data. The data will be written on the TFRecord file entry by entry.
 3. Create a feature dictionary (`tf.train.Features`) with the desired keys. The corresponding values will be the data represented as TensorFlow feature types (`_bytes_feature`, `_int64_feature`, ...).
 4. Create an example (`tf.train.Example`) from the feature dictionary which will be a record on the TFRecord file.
 5. Serialize the examples.
 6. Write on the file.
 7. Close the file.

In [ ]:
filename_label = [('imgs/orange-cat.jpg', 'orange-cat'),
                  ('imgs/gray-cat.jpg', 'gray-cat')]

with tf.io.TFRecordWriter('cats.tfrecords') as writer:
    for img_path, label in filename_label:
        cat_array = io.imread(img_path)
        
        features_dict = {
            'image': _bytes_feature(cat_array.tobytes()),
            'label': _bytes_feature(label.encode(encoding='utf-8')),
            'height': _int64_feature(cat_array.shape[0]),
            'width': _int64_feature(cat_array.shape[1])
        }
                
        example = tf.train.Example(features=tf.train.Features(feature=features_dict))
        
        example_serialized = example.SerializeToString()

        writer.write(example_serialized)

***

### 3. Read from TFRecord file to Dataset object:

Steps:
 1. Load the TFRecord file to a dataset object with `the function tf.data.TFRecordDataset(tfrecords_filename)`.
 2. Create a function to parse the data from the TFRecord file into python or TensorFlow types and apply it through a map to the dataset.
 3. Do the normal operations to a dataset to define the input pipeline: maps, filters, batch, repeat, etc.
 
Parser function.
 1. Define a parser (`tf.io.parse_single_example`) from the serialized example and the features (keys) that will be loaded specifying their respective types (not all keys saved on the records need to be loaded).
 2. Return the the decoded data. 

In [ ]:
def decode(serialized_example):
    """Parse an image and label from the given `serialized_example`."""

    example = tf.io.parse_single_example(
        serialized_example,
        features={
            'height': tf.io.FixedLenFeature((), tf.int64),
            'width':  tf.io.FixedLenFeature((), tf.int64),
            'image':  tf.io.FixedLenFeature((), tf.string),
            'label':  tf.io.FixedLenFeature((), tf.string),
        })

    width = example['width']
    height = example['height']
    label = example['label']
    image = tf.io.decode_raw(example['image'], tf.uint8)
    image = tf.reshape(image, (height, width, 3))

    return image, label

In [ ]:
dataset = tf.data.TFRecordDataset('cats.tfrecords')
dataset = dataset.map(decode)
dataset = dataset.batch(1)

In [ ]:
for image, label in dataset:
    io.imshow(image.numpy()[0])
    tf.print(label[0])
    plt.xticks([])
    plt.yticks([])
    plt.show()